## A* SEARCH ALGORITHM
### An interactive visualization of the A* algorithm using PyGame in Python

#### Importing necessary libraries

In [1]:
import pygame
import math
from queue import PriorityQueue

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Setting the visualization environment using PyGame

In [2]:
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption('A*')

## Node Class 
##### Defigning a node class so that every square on the grade can be seen as a node in a connected graph. This way we can track of where the node is (row, column), all it's neighbors, node properties (width, color, etc) so we can draw the "graph" on screen, etc.

In [3]:
#COLORS

MINT = (199, 236, 143)
L_GREEN = (75, 178, 173)
DARK_BLUE = (47, 69, 102)
WHITE= (255, 255, 255)
TORQUIZE = (64, 224, 208)
YELLOW =(255, 255, 0)
RED = (255, 0, 0)
GREY =(230, 230, 230)

class Node:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
        
    def get_pos(self):
        return self.row, self.col
    
    def is_closed(self):
        return self.color == TORQUIZE
    
    def is_open(self):
        return self.color == MINT
    
    def is_barrier(self):
        return self.color == DARK_BLUE
    
    def is_start(self):
        return self.color == L_GREEN
    
    def is_end(self):
        return self.color == RED
    
    def reset(self):
        self.color = WHITE
    
    def make_close(self):
        self.color = TORQUIZE
        
    def make_open(self):
        self.color = MINT
        
    def make_barrier(self):
        self.color = DARK_BLUE
        
    def make_start(self):
        self.color = L_GREEN
    
    def make_end(self):
        self.color = RED
        
    def make_path(self):
        self.color = YELLOW
        
    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
        
    def update_neighbors(self, grid):
        self.neighbors = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): #CHECK DOWN NEIGHBOR
            self.neighbors.append(grid[self.row + 1][self.col])
            
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): #CHECK UP NEIGHBOR
            self.neighbors.append(grid[self.row - 1][self.col])
            
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): #CHECK RIGHT NEIGHBOR
            self.neighbors.append(grid[self.row][self.col + 1])
            
        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): #CHECK LEFT NEIGHBOR
            self.neighbors.append(grid[self.row][self.col - 1])


#### The update_neighbors method checks if an adjacent node of the current node is accessible (not a barrier)

### Creating the main grid

In [4]:
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(i, j, gap, rows)
            grid[i].append(node)
            
    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))

def draw(win, grid, rows, width):
    win.fill(WHITE)

    for row in grid:
        for node in row:
            node.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()

def get_clicked_position(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    
    return row, col


def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

### The heuristic function calculates the distance betwwen two points (p1, p2) using Mangattan distance (L distance)
<img src="heuristic_ex.jpg">

In [5]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

## A* algorithm
#### references :

1- https://en.wikipedia.org/wiki/A*_search_algorithm#Pseudocode 

2- https://www.youtube.com/watch?v=ySN5Wnu88nE&t=750s


In [6]:
def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node : float('inf') for row in grid for node in row}
    g_score[start] = 0
    
    f_score = {node : float('inf') for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    
    open_set_hash = {start}
    
    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            end.make_end()
            reconstruct_path(came_from, end, draw)
            start.make_start()
            
            
            
            return True
        
        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1
            
            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()
        draw()
        
        if current != start:
            current.make_close()
    
    return False

#### Setting the main "game loop"... this is pygame after all.

In [7]:
def main(win, width):
    ROWS = 50
    grid = make_grid(ROWS, width)
    
    start = None
    end = None
    run = True
    
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                

            
            if pygame.mouse.get_pressed()[0]: # LEFT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_position(pos, ROWS, width)
                node = grid[row][col]
                if not start and node != end:
                    start = node
                    start.make_start()

                elif not end and node != start:
                    end = node
                    end.make_end()

                elif node != end and node != start:
                    node.make_barrier()

            elif pygame.mouse.get_pressed()[2]: # RIGHT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_position(pos, ROWS, width)
                node = grid[row][col]
                node.reset()
                if node == start:
                    start = None
                elif node == end:
                    end = None
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for node in row:
                            node.update_neighbors(grid)
                    algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)
            
                if event.key == pygame.K_r:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)
                    
    pygame.quit()
    
main(WIN, WIDTH)    